<a href="https://colab.research.google.com/github/Ashish8349/stackoverflow-survey-2019/blob/main/Stackoverflow_survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dash
!pip install jupyter_dash

     |████████████████████████████████| 1.1 MB 7.6 MB/s 
     |████████████████████████████████| 3.7 MB 40.2 MB/s 
     |████████████████████████████████| 83 kB 2.9 MB/s 
     |████████████████████████████████| 1.8 MB 44.6 MB/s 
     |████████████████████████████████| 357 kB 60.5 MB/s 
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl size=1100489 sha256=67d966fa5eb890070b065b224e085ddc063602b67d998af27e9b051f13861522
  Stored in directory: /root/.cache/pip/wheels/59/a4/f1/a7140afcad8dbaa3b1b12ef80843d29403d6ecca1c2f397948
  Created wheel for dash-core-components: filename=dash_core_components-1.17.1-py3-none-any.whl size=3738995 sha256=ca7dc7ae472d8ae019ba2bb3453c2c8d9d5519df1079c431fea150f7923e5847
  Stored in directory: /root/.cache/pip/wheels/ca/5b/fb/a135a7bf8e21223e4104aa5526a5173a20b64fe7fcd055b999
  Created wheel for dash-html-components: filename=dash_html_components-1.1.4-py3-none-any.whl size=319771 sha256=29bf4727e0307264f53458df1babdb3f21ee65604527d2f26dceca5

In [5]:
import pandas as pd
import plotly as plt
import plotly.express as px
import dash
import dash_html_components as html
import dash_core_components as dcc
from jupyter_dash import JupyterDash

In [7]:
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv"
df = pd.read_csv(dataset_url)
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,...,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,100 to 499 employees,"Developer, full-stack",3,16,Less than 1 year,Very satisfied,Slightly satisfied,Very confident,No,Not sure,I am not interested in new job opportunities,Less than a year ago,"Write code by hand (e.g., on a whiteboard);Int...",No,"Languages, frameworks, and other technologies ...",I was preparing for a job search,USD,United States dollar,61000.0,Yearly,61000.0,80.0,There's no schedule or spec; I work on what se...,NaN,Less than once per month / Never,Home,A little below average,No,NaN,...,MySQL;SQLite,MySQL;SQLite,Linux;Windows,Linux;Windows,NaN,NaN,.NET,.NET,Eclipse;Vim;Visual Studio;Visual Studio Code,Windows,I do not use containers,Not at all,"Useful for decentralized currency (i.e., Bitcoin)",Yes,SIGH,Yes,Reddit,In real life (in person),Username,2014,Daily or almost daily,Find answers to specific questions;Pass the ti...,1-2 times per week,Stack Overflow was much faster,31-60 minutes,Yes,Less than once per month or monthly,Yes,"No, and I don't know what those are","No, not really",Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
1,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,10 to 19 employees,"Database administrator;Developer, back-end;Dev...",12,11,4,Slightly satisfied,Slightly satisfied,Somewhat confident,No,Not sure,"I’m not actively looking, but I am open to new...",Less than a year ago,Write any code;Interview with people in peer r...,Yes,Financial performance or funding status of the...,I was preparing for a job search,NZD,New Zealand dollar,138000.0,Yearly,95179.0,32.0,There is a schedule and/or spec (made by me or...,Being tasked with non-development work;Inadequ...,Less than once per month / Never,Office,A little above average,"Yes, because I see value in code review",12.0,...,DynamoDB;PostgreSQL;SQLite,PostgreSQL;Redis;SQLite,AWS;Docker;Heroku;Linux;MacOS;Slack,AWS;Docker;Heroku;Linux;MacOS;Slack;Other(s):,Express;Ruby on Rails;Other(s):,Express;Ruby on Rails;Other(s):,Node.js;Unity 3D,Node.js,Vim,MacOS,Development;Testing;Production,Not at all,An irresponsible use of resources,No,SIGH,Yes,Twitter,In real life (in person),Username,2013,Daily or almost daily,Find answers to specific questions;Contribute ...,3-5 times per week,They were about the same,NaN,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,NaN,23.0

**Explore the data and get familiar with dataset**

In [12]:
#df.shape
#print(df.dtypes)
#df.info()
df.describe()

,Respondent,CompTotal,ConvertedComp,WorkWeekHrs,CodeRevHrs,Age
count,11552.000000,1.073700e+04,1.073000e+04,11427.000000,9083.000000,11255.000000
mean,12362.212517,7.499932e+05,1.313340e+05,42.051851,4.762829,30.772394
std,7271.939210,9.639522e+06,2.943245e+05,24.528561,4.548401,7.392780
min,4.000000,0.000000e+00,0.000000e+00,3.000000,0.000000,16.000000
25%,6011.500000,2.500000e+04,2.672700e+04,40.000000,2.000000,25.000000
50%,12323.500000,6.500000e+04,5.774400e+04,40.000000,4.000000,29.000000
75%,18686.500000,1.200000e+05,1.000000e+05,43.000000,5.000000,35.000000
max,25142.000000,7.000000e+08,2.000000e+06,1012.000000,99.000000,99.000000


 **Performing data wrangling :-
identify duplicate values and remove those,identify missing values and either impute those or remove.
Normalize data in the dataset.**

In [13]:
df[df.duplicated()].shape       #identify the duplicates
#df[df.duplicated('Respondent')].shape

(154, 85)

In [14]:
df = df.drop_duplicates()    # Removing the duplicates
df.shape

(11398, 85)

In [ ]:
df[df.duplicated()].shape   # Check if any duplicate exist

(0, 85)

In [17]:
df.isnull().sum()   # Check for null values in dataframe
#df.isnull().sum().sum()

Respondent        0
MainBranch        0
Hobbyist          0
OpenSourcer       0
OpenSource       81
               ... 
Sexuality       542
Ethnicity       675
Dependents      140
SurveyLength     19
SurveyEase       14
Length: 85, dtype: int64

In [18]:
df['WorkLoc'].isnull().value_counts() # Check how many null values are there in column 'WorkLoc'

False    11366
True        32
Name: WorkLoc, dtype: int64

In [ ]:
df['WorkLoc'].value_counts()   # Check for more frequent value in column 'WorkLoc'

Office                                            6806
Home                                              3589
Other place, such as a coworking space or cafe     971
Name: WorkLoc, dtype: int64

In [ ]:
df['WorkLoc'].fillna('Office', inplace=True)  # Fill null values with 'Office'
df['WorkLoc'].isnull().value_counts()

False    11398
Name: WorkLoc, dtype: int64

In [ ]:
df['CompFreq'].value_counts() # How compensation are paid to respondents

Yearly     6073
Monthly    4788
Weekly      331
Name: CompFreq, dtype: int64

In [ ]:
df[['Respondent','CompFreq','CompTotal']].head(20)

,Respondent,CompFreq,CompTotal
0,4,Yearly,61000.0
1,9,Yearly,138000.0
2,13,Yearly,90000.0
3,16,Monthly,29000.0
4,17,Yearly,90000.0
5,19,Monthly,9500.0
6,20,Monthly,3000.0
7,22,Yearly,103000.0
8,23,Yearly,69000.0
9,24,Monthly,8000.0


**Normalize the data in 'CompTotal' column total compensation on yearly basis**

In [ ]:
df.loc[df['CompFreq'] == 'Yearly','NormalizedAnnualCompensation'] = df['CompTotal']
df.loc[df['CompFreq'] == 'Monthly','NormalizedAnnualCompensation'] = df['CompTotal']*12
df.loc[df['CompFreq'] == 'Weekly','NormalizedAnnualCompensation'] = df['CompTotal']*52
df[['Respondent','CompFreq','CompTotal','NormalizedAnnualCompensation']].head(20)

,Respondent,CompFreq,CompTotal,NormalizedAnnualCompensation
0,4,Yearly,61000.0,61000.0
1,9,Yearly,138000.0,138000.0
2,13,Yearly,90000.0,90000.0
3,16,Monthly,29000.0,348000.0
4,17,Yearly,90000.0,90000.0
5,19,Monthly,9500.0,114000.0
6,20,Monthly,3000.0,36000.0
7,22,Yearly,103000.0,103000.0
8,23,Yearly,69000.0,69000.0
9,24,Monthly,8000.0,96000.0


In [19]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m5_survey_data_demographics.csv"
url2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m5_survey_data_technologies_normalised.csv"

In [20]:
df1 = pd.read_csv(url)
df2 = pd.read_csv(url2)

In [25]:
#df2.drop(0, axis=0, inplace=True)
#df2.head(10)
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,...,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,100 to 499 employees,"Developer, full-stack",3,16,Less than 1 year,Very satisfied,Slightly satisfied,Very confident,No,Not sure,I am not interested in new job opportunities,Less than a year ago,"Write code by hand (e.g., on a whiteboard);Int...",No,"Languages, frameworks, and other technologies ...",I was preparing for a job search,USD,United States dollar,61000.0,Yearly,61000.0,80.0,There's no schedule or spec; I work on what se...,NaN,Less than once per month / Never,Home,A little below average,No,NaN,...,MySQL;SQLite,MySQL;SQLite,Linux;Windows,Linux;Windows,NaN,NaN,.NET,.NET,Eclipse;Vim;Visual Studio;Visual Studio Code,Windows,I do not use containers,Not at all,"Useful for decentralized currency (i.e., Bitcoin)",Yes,SIGH,Yes,Reddit,In real life (in person),Username,2014,Daily or almost daily,Find answers to specific questions;Pass the ti...,1-2 times per week,Stack Overflow was much faster,31-60 minutes,Yes,Less than once per month or monthly,Yes,"No, and I don't know what those are","No, not really",Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
1,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,10 to 19 employees,"Database administrator;Developer, back-end;Dev...",12,11,4,Slightly satisfied,Slightly satisfied,Somewhat confident,No,Not sure,"I’m not actively looking, but I am open to new...",Less than a year ago,Write any code;Interview with people in peer r...,Yes,Financial performance or funding status of the...,I was preparing for a job search,NZD,New Zealand dollar,138000.0,Yearly,95179.0,32.0,There is a schedule and/or spec (made by me or...,Being tasked with non-development work;Inadequ...,Less than once per month / Never,Office,A little above average,"Yes, because I see value in code review",12.0,...,DynamoDB;PostgreSQL;SQLite,PostgreSQL;Redis;SQLite,AWS;Docker;Heroku;Linux;MacOS;Slack,AWS;Docker;Heroku;Linux;MacOS;Slack;Other(s):,Express;Ruby on Rails;Other(s):,Express;Ruby on Rails;Other(s):,Node.js;Unity 3D,Node.js,Vim,MacOS,Development;Testing;Production,Not at all,An irresponsible use of resources,No,SIGH,Yes,Twitter,In real life (in person),Username,2013,Daily or almost daily,Find answers to specific questions;Contribute ...,3-5 times per week,They were about the same,NaN,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,NaN,23.0

In [ ]:
#Top 10 Language 

lang = df2[['Respondent','LanguageWorkedWith']].groupby(by='LanguageWorkedWith', axis=0).count().reset_index()
lang.sort_values(by='Respondent',ascending=False,inplace=True)
top_10_lang = lang.iloc[:10]

#Top 10 Desired language

desired_lang = df2[['Respondent','LanguageDesireNextYear']].groupby(by='LanguageDesireNextYear', axis=0).count().reset_index()
desired_lang.sort_values(by='Respondent',ascending=False,inplace=True)
top_10_desired_lang = desired_lang.iloc[:10]

#Top 10 database

db = df2[['Respondent','DatabaseWorkedWith']].groupby(by='DatabaseWorkedWith', axis=0).count().reset_index()
db.sort_values(by='Respondent',ascending=False,inplace=True)
top_10_db = db.iloc[:10]

#Top 10 desired database

desired_db = df2[['Respondent','DatabaseDesireNextYear']].groupby(by='DatabaseDesireNextYear', axis=0).count().reset_index()
desired_db.sort_values(by='Respondent',ascending=False,inplace=True)
top_10_desired_db = desired_db.iloc[:10]

#Top 10 webframework

Web_FrameWork = df2[['Respondent','WebFrameWorkedWith']].groupby(by='WebFrameWorkedWith',axis=0).count().reset_index()
Web_FrameWork.sort_values(by='Respondent',ascending=False,inplace=True)
top_10_Web_FrameWork = Web_FrameWork.iloc[:10]

#Top 10 desired webframework

desired_Web_FrameWork = df2[['Respondent','WebFrameDesireNextYear']].groupby(by='WebFrameDesireNextYear',axis=0).count().reset_index()
desired_Web_FrameWork.sort_values(by='Respondent',ascending=False,inplace=True)
top_10_desired_Web_FrameWork = desired_Web_FrameWork.iloc[:10]

#Number of Respondents by Age
respondents_by_age = df1[['Respondent','Age']].groupby(by='Age',axis=0).count().reset_index()


In [ ]:
top_10_lang_fig = px.bar(top_10_lang.sort_values(by='Respondent'), x='Respondent',y='LanguageWorkedWith', orientation='h', title="Top 10 Languages",text='Respondent')
top_10_desired_lang_fig = px.bar(top_10_desired_lang.sort_values(by='Respondent'), x='Respondent',y='LanguageDesireNextYear', orientation='h', 
                                 title="Top 10 Desired Languages",text='Respondent')
top_10_db_fig = px.bar(top_10_db, x='DatabaseWorkedWith',y='Respondent', title="Top 10 Databases",text='Respondent')
top_10_desired_db_fig = px.bar(top_10_desired_db, x='DatabaseDesireNextYear',y='Respondent', title="Top 10 Desired Databases",text='Respondent')

respondents_by_age_fig = px.line(respondents_by_age,x='Age', y='Respondent')

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[html.H1('Stackoverflow survey 2019', 
                                         style={'textAlign': 'center',
                                                'color': '#503D36',
                                                 'font-size': 40}),
                                html.Div([
                                        html.Div(dcc.Graph(figure=top_10_lang_fig)),
                                        html.Div(dcc.Graph(figure=top_10_desired_lang_fig))
                                ], style={'display': 'flex'}),

                                html.Div([
                                        html.Div(dcc.Graph(figure=top_10_db_fig)),
                                        html.Div(dcc.Graph(figure=top_10_desired_db_fig))
                                ], style={'display': 'flex'}),

                                html.Div(dcc.Graph(figure=respondents_by_age_fig))])
                                
                    

if __name__ == '__main__':
    app.run_server(mode='external', host="localhost")

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
print("To test new changes")

To test new changes
